In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import date
from datetime import timedelta

from yahoofinancials import YahooFinancials

from scipy.stats import norm

import warnings
warnings.filterwarnings('ignore')

In [2]:
end = '2021-07-22'
start = '2016-07-20'

In [3]:
weight_df = pd.read_csv('../data/WeightsDF.csv')
weight_df.set_index('Portfolio', inplace=True)

In [4]:
weight_df

,IRCTC,NESTLEIND,INDIAMART,TITAN,INFY,TATACONSUM,TCS,DIXON,HINDUNILVR,ATGL,DIVISLAB,POLYCAB
Portfolio,,,,,,,,,,,,
Portfolio 1,0.26313,0.03165,0.33668,0.00018,0.01454,0.02776,0.01332,NaN,0.01153,0.28391,0.01730,NaN
Portfolio 2,0.24072,0.01554,0.30410,NaN,0.00401,0.00021,0.00017,0.08996,NaN,0.24679,0.00722,0.09128


In [5]:
stocks = np.array(weight_df.columns).tolist()

i1 = 0
i2 = 1
portfolio1 = []
portfolio2 = []
for stock in stocks:
    if weight_df[stock][i1] > 0:
        portfolio1.append(stock)
    if weight_df[stock][i2] > 0:
        portfolio2.append(stock)

In [6]:
portfolio_ticker = []
for stock in stocks:
    portfolio_ticker.append(stock+'.NS')

In [7]:
for stock, stock_symbol in zip(stocks, portfolio_ticker):
    yf = YahooFinancials(stock_symbol)
    data = yf.get_historical_price_data(start, end, 'daily')
    globals()[stock] = pd.DataFrame(data[stock_symbol]['prices'])
    globals()[stock] = globals()[stock].drop('date', axis=1).set_index('formatted_date')

In [8]:
yf = YahooFinancials('^NSEI')
data = yf.get_historical_price_data(start, end, 'daily')
nifty_50 = pd.DataFrame(data['^NSEI']['prices'])
nifty_50 = nifty_50.drop('date', axis=1).set_index('formatted_date')

In [9]:
nifty_50_daily = nifty_50['adjclose'].pct_change().dropna()

In [10]:
closing_df = pd.DataFrame(index=TCS.index, columns=stocks)
for stock in stocks:
    closing_df[stock] = globals()[stock]['adjclose']
    
closing_df['nifty_50'] = nifty_50['adjclose']

In [11]:
closing_df.index = pd.to_datetime(closing_df.index)

In [12]:
closing_df

,IRCTC,NESTLEIND,INDIAMART,TITAN,INFY,TATACONSUM,TCS,DIXON,HINDUNILVR,ATGL,DIVISLAB,POLYCAB,nifty_50
formatted_date,,,,,,,,,,,,,
2016-07-20,NaN,6158.305664,NaN,400.601959,472.743866,123.635284,1129.042114,NaN,842.082458,NaN,1143.102295,NaN,8565.849609
2016-07-21,NaN,6336.754883,NaN,397.782928,471.347107,121.741737,1134.065430,NaN,845.007202,NaN,1144.402954,NaN,8510.099609
2016-07-22,NaN,6531.118652,NaN,398.268921,468.095337,121.464615,1138.160645,NaN,838.646851,NaN,1160.588379,NaN,8541.200195
2016-07-25,NaN,6575.546387,NaN,401.233795,471.761658,123.265816,1161.004517,NaN,842.128784,NaN,1160.106689,NaN,8635.650391
2016-07-26,NaN,6610.284668,NaN,402.448944,475.188019,121.141327,1158.123901,NaN,838.182678,NaN,1143.054077,NaN,8590.650391
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-14,2266.399902,17539.699219,7397.700195,1706.256836,1576.900024,768.200012,3207.550049,4593.100098,2414.850098,881.250000,4623.200195,1903.250000,15853.950195
2021-07-15,2268.300049,17621.849609,7297.950195,1697.477905,1580.800049,772.349976,3202.949951,4645.500000,2412.350098,892.900024,4607.299805,1899.250000,15924.200195
2021-07-16,2426.649902,17654.199219,7228.950195,1691.292725,1555.449951,763.000000,3194.800049,4646.299805,2417.399902,896.549988,4757.600098,1936.099976,15923.400391


In [13]:
daily_return = closing_df.pct_change()
monthly_return = closing_df.resample('M').ffill().pct_change() 

In [14]:
daily_return

,IRCTC,NESTLEIND,INDIAMART,TITAN,INFY,TATACONSUM,TCS,DIXON,HINDUNILVR,ATGL,DIVISLAB,POLYCAB,nifty_50
formatted_date,,,,,,,,,,,,,
2016-07-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-21,NaN,0.028977,NaN,-0.007037,-0.002955,-0.015316,0.004449,NaN,0.003473,NaN,0.001138,NaN,-0.006508
2016-07-22,NaN,0.030672,NaN,0.001222,-0.006899,-0.002276,0.003611,NaN,-0.007527,NaN,0.014143,NaN,0.003655
2016-07-25,NaN,0.006802,NaN,0.007444,0.007832,0.014829,0.020071,NaN,0.004152,NaN,-0.000415,NaN,0.011058
2016-07-26,NaN,0.005283,NaN,0.003029,0.007263,-0.017235,-0.002481,NaN,-0.004686,NaN,-0.014699,NaN,-0.005211
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-14,-0.003539,-0.008603,0.023662,-0.008061,0.020680,-0.006017,0.008470,0.001504,-0.008967,-0.045956,0.003723,-0.005539,0.002631
2021-07-15,0.000838,0.004684,-0.013484,-0.005145,0.002473,0.005402,-0.001434,0.011408,-0.001035,0.013220,-0.003439,-0.002102,0.004431
2021-07-16,0.069810,0.001836,-0.009455,-0.003644,-0.016036,-0.012106,-0.002544,0.000172,0.002093,0.004088,0.032622,0.019402,-0.000050


In [15]:
monthly_return

,IRCTC,NESTLEIND,INDIAMART,TITAN,INFY,TATACONSUM,TCS,DIXON,HINDUNILVR,ATGL,DIVISLAB,POLYCAB,nifty_50
formatted_date,,,,,,,,,,,,,
2016-07-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-08-31,NaN,-0.090601,NaN,-0.006882,-0.035387,0.002856,-0.041232,NaN,-0.005962,NaN,0.103463,NaN,0.017098
2016-09-30,NaN,-0.014205,NaN,-0.055914,0.000193,-0.004634,-0.032055,NaN,-0.053650,NaN,-0.023970,NaN,-0.019923
2016-10-31,NaN,0.085949,NaN,-0.054543,-0.026883,0.002507,-0.010557,NaN,-0.036124,NaN,-0.008444,NaN,0.003118
2016-11-30,NaN,-0.099638,NaN,-0.133985,-0.022005,-0.115041,-0.051132,NaN,0.017480,NaN,-0.082741,NaN,-0.047870
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-31,-0.001506,0.066056,-0.099793,0.107435,0.091558,0.048839,0.097968,-0.074974,0.140452,0.876708,0.077220,0.031748,0.011119
2021-04-30,0.019579,-0.044697,0.027229,-0.042617,-0.010014,0.047895,-0.044747,0.144234,-0.031976,0.197139,0.121329,0.063426,-0.004057
2021-05-31,0.062409,0.085001,-0.072843,0.070124,0.040196,-0.008439,0.045774,-0.041173,-0.005820,0.172149,0.032407,0.141236,0.065046


In [16]:
monthly_volatility = daily_return.rolling(window=21).std() * np.sqrt(21)

In [17]:
monthly_volatility

,IRCTC,NESTLEIND,INDIAMART,TITAN,INFY,TATACONSUM,TCS,DIXON,HINDUNILVR,ATGL,DIVISLAB,POLYCAB,nifty_50
formatted_date,,,,,,,,,,,,,
2016-07-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-14,0.079298,0.039923,0.052422,0.042695,0.051741,0.039674,0.051913,0.073706,0.046639,0.175828,0.050680,0.060775,0.021705
2021-07-15,0.078963,0.036583,0.052828,0.042723,0.051729,0.034049,0.051728,0.072076,0.046203,0.177109,0.050757,0.052338,0.021060
2021-07-16,0.099139,0.036143,0.052220,0.042819,0.054611,0.035249,0.049561,0.068982,0.046069,0.175835,0.058440,0.055136,0.020324


In [18]:
monthly_closing_df = closing_df.resample('M').mean()
monthly_closing_df

,IRCTC,NESTLEIND,INDIAMART,TITAN,INFY,TATACONSUM,TCS,DIXON,HINDUNILVR,ATGL,DIVISLAB,POLYCAB,nifty_50
formatted_date,,,,,,,,,,,,,
2016-07-31,NaN,6506.459900,NaN,403.111191,471.483467,124.870709,1158.323608,NaN,843.289497,NaN,1150.496536,NaN,8595.506226
2016-08-31,NaN,6277.007968,NaN,396.994168,459.801376,130.304664,1189.948952,NaN,855.071275,NaN,1202.466675,NaN,8638.911877
2016-09-30,NaN,5940.852319,NaN,400.254443,455.942860,131.627499,1091.926129,NaN,846.042593,NaN,1274.640894,NaN,8783.325098
2016-10-31,NaN,6270.705539,NaN,377.665831,450.568503,139.842376,1086.221992,NaN,793.631297,NaN,1222.873725,NaN,8669.027778
2016-11-30,NaN,5787.533133,NaN,325.481855,419.803230,115.396306,1009.394148,NaN,773.441185,NaN,1161.554112,NaN,8250.749977
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-31,1875.790475,16571.851981,8105.533319,1478.363031,1334.739705,616.569420,3048.471924,3908.867618,2237.205357,763.788443,3474.333310,1352.481253,14835.099981
2021-04-30,1698.952637,16929.548623,8130.702611,1516.995059,1364.861270,663.143478,3155.891859,3796.128945,2388.368742,1119.392915,3781.892128,1422.196636,14613.852539
2021-05-31,1815.020001,17125.034961,7399.907495,1495.716888,1339.242767,644.606570,3084.590381,3985.697522,2352.562817,1306.864575,4069.825012,1583.053711,14983.525000


In [19]:
nifty_std = monthly_return['nifty_50'].std()

mstd = monthly_return.copy()
mstdl = mstd.corr(method='pearson')

In [20]:
monthly_stock_beta = pd.Series(index=stocks)

for stock in stocks:
    monthly_stock_beta[stock] = (mstdl['nifty_50'][stock] * mstd[stock].std()) / nifty_std

In [21]:
monthly_stock_beta

IRCTC         1.413335
NESTLEIND     0.232141
INDIAMART     0.958099
TITAN         0.944141
INFY          0.587275
TATACONSUM    0.786644
TCS           0.553969
DIXON         1.260711
HINDUNILVR    0.160515
ATGL          2.375908
DIVISLAB      0.331044
POLYCAB       1.150129
dtype: float64

In [22]:
w1 = weight_df.iloc[0].dropna()
w2 = weight_df.iloc[1].dropna()

In [23]:
metrics = ['Return','Volatility','Sharpe Ratio', 'VaR', 'Portfolio Beta', 'Treynor Ratio', 'Jensen Alpha']
metrics_df = pd.DataFrame(columns=metrics)

In [24]:
risk_free_rate = 0.0602
alpha = 0.01
days = 21
market_return = nifty_50_daily.mean() * days 

In [25]:
# P1

r1 = np.zeros(len(metrics))
sb1 = monthly_stock_beta[portfolio1].copy()
pb1 = 0
tdf1 = monthly_return[portfolio1].copy()
twl1 = w1
tcovdf1 = tdf1.cov()

pr1 = np.sum(tdf1.mean() * twl1) * days
pstd1 = np.sqrt(np.dot(twl1.T,np.dot(tcovdf1, twl1))) * np.sqrt(days)

r1[0] = pr1
r1[1] = pstd1

# SHARPE RATIO => (portfolio_return - risk_free_rate) / volatility
r1[2] = (r1[0] - risk_free_rate) / r1[1]

# VaR => return - (Z * volatility)
r1[3] = abs(pr1 - (pstd1 * norm.ppf(1 - alpha)))

# PORTFOLIO BETA => sum(weight * beta)
for j in range(len(twl1)):
    pb1 += twl1[j] * sb1[j]
r1[4] = pb1

# TREYNOR RATIO => (portfolio_return - risk_free_rate) / portfolio_beta
r1[5] = (r1[0] - risk_free_rate) / r1[4]

# JENSEN ALPHA => portfolio_return - (risk_free_rate + portfolio_beta * (market_return - risk_free_rate))
r1[6] = r1[0] - (risk_free_rate + r1[4] * (market_return - risk_free_rate))   

r1 = pd.Series(r1, index=metrics)
metrics_df = metrics_df.append(r1, ignore_index=True)

In [26]:
# P2

r2 = np.zeros(len(metrics))
sb2 = monthly_stock_beta[portfolio2].copy()
pb2 = 0
tdf2 = monthly_return[portfolio1].copy()
twl2 = w2
tcovdf2 = tdf2.cov()

pr2 = np.sum(tdf2.mean() * twl2) * days
pstd2 = np.sqrt(np.dot(twl2.T,np.dot(tcovdf2, twl2))) * np.sqrt(days)

r2[0] = pr2
r2[1] = pstd2

# SHARPE RATIO => (portfolio_return - risk_free_rate) / volatility
r2[2] = (r2[0] - risk_free_rate) / r2[1]

# VaR => return - (Z * volatility)
r2[3] = abs(pr2 - (pstd2 * norm.ppf(1 - alpha)))

# PORTFOLIO BETA => sum(weight * beta)
for j in range(len(twl2)):
    pb2 += twl2[j] * sb2[j]
r2[4] = pb2

# TREYNOR RATIO => (portfolio_return - risk_free_rate) / portfolio_beta
r2[5] = (r2[0] - risk_free_rate) / r2[4]

# JENSEN ALPHA => portfolio_return - (risk_free_rate + portfolio_beta * (market_return - risk_free_rate))
r2[6] = r2[0] - (risk_free_rate + r2[4] * (market_return - risk_free_rate))   

r2 = pd.Series(r2, index=metrics)
metrics_df = metrics_df.append(r2, ignore_index=True)

In [27]:
metrics_df

,Return,Volatility,Sharpe Ratio,VaR,Portfolio Beta,Treynor Ratio,Jensen Alpha
0,1.569770,0.478594,3.154174,0.456393,1.421858,1.061689,1.578602
1,1.362721,0.337525,3.859034,0.577520,1.444935,0.901439,1.372673


In [28]:
m = pd.read_csv('../data/MetricsDF.csv')
m

,Portfolio,Return,Volatility,Sharpe Ratio,VaR,Portfolio Beta,Treynor Ratio,Jensen Alpha
0,Portfolio 1,0.900515,0.325621,2.580652,0.143007,0.892660,0.941361,0.760400
1,Portfolio 2,0.890909,0.312729,2.656319,0.163392,0.923608,0.899418,0.748023
